# Aplicando e Definindo o Modelo

Formatando o kernel e criando as camadas

In [4]:
from tensorflow.keras import models, layers, activations, initializers

model = models.Sequential([
    layers.Input(shape=(600, 600, 3)),   # definindo o modelo

    layers.Resizing(600, 600),           
    layers.Rescaling(1./255),  # salva pixels na mesma escala
    layers.RandomRotation((-0.2, 0.2)),

    # usado para identificar padrões em imagens
    layers.Conv2D(10, kernel_size=(3,3), strides=(1,1), padding='same'),     
     # reduz o tamanho da imagem pegando apenas o valor máximo dentro de grids, evita o overfitting
    layers.MaxPooling2D((2,2)),          

    # transforma uma matriz em um vetor único 
    layers.Flatten(),     

     # camada com N neurônio, com ativação relu (para valores negativos, a saída é zero e positivos passam direto)
    layers.Dense(
        128,
        activations.relu,
        kernel_initializer=initializers.RandomNormal()), 

    # elimina aleatóriamente 20% alguns neurônios durante a fase de treinamento
    layers.Dropout(0.2),  

    # sigmoid: transforma qualquer número em um valor entre 0 e 1
    layers.Dense(
        128,
        activations.sigmoid,
        kernel_initializer=initializers.RandomNormal()), 

    # usado quando a rede precisa escolher uma classe. Pega um vetor de números e transforma em uma probabilidade que somam 1
    layers.Dense(
        64,
        activations.softmax,
        kernel_initializer=initializers.RandomNormal()), 

    #  Gelu : ReLU suave , probabilística
    # layers.Dense(
    #     64,
    #     activations.gelu,
    #     kernel_initializer=initializers.RandomNormal()),
])

# Otimização e Métricas
Calculando o erro (Adam - Mistura de momentum + adaptativo. É o mais usado, aprende mais rápido e estável)

In [5]:
from tensorflow.keras import optimizers, losses, metrics

lr = 0.001

model.compile(
    optimizer = optimizers.Adam(
        learning_rate = lr
    ),
    loss = losses.SparseCategoricalCrossentropy(),  # mede a acurácia considerando rótulos inteiros (não one-hot)
    metrics = [ metrics.sparse_categorical_accuracy ]
)

# Lendo o Dataset

lendo as imagens de teste e de treino do dataset

In [13]:
from tensorflow.keras import utils

path = './Data'
batch_size = 100 

train = utils.image_dataset_from_directory(
    directory = path,
    shuffle = True,
    subset = 'training',
    validation_split = 0.1,
    seed = 1,
    image_size = (600, 600),
    batch_size = batch_size
)

test = utils.image_dataset_from_directory(
    directory = path,
    shuffle = True,
    subset = 'validation',
    validation_split = 0.1,
    seed = 1,
    image_size = (600, 600),
    batch_size = batch_size
)


Found 2000 files belonging to 2 classes.
Using 1800 files for training.
Found 2000 files belonging to 2 classes.
Using 200 files for validation.


# Treinando o Modelo

In [ ]:
from tensorflow.keras import callbacks

patience = 3  # tolerância para evitar overfitting, 